# Takip Algoritmaları 
- BOOSTING Takip Algoritması 
- TLD (Tracking, learning and detection) Takip Algoritması
- MOSSE (Minimum Output Sum of Squared Error) Takip Algoritması
- CSRT Takip Algoritması
- KCF (Kernelized Correlation Filters) Takip Algoritması
- MIL (Multiple Instance Learning) Takip Algoritması

## BOOSTING Takip Algoritması
- AdaBoost'un çevrimiçi bir sürümüne dayanmaktadır
- Bu sınıflandırıcı, nesnenin olumlu ve olumsuz örnekleriyle çalışma zamanında eğitilmelidir.
- Kullanıcı(veya başka bir nesne algılama algoritması) tarafından sağlanan ilk sınırlayıcı kutu, nesne için olumlu bir örnek olarak alınır ve sınırlayıcı kutnun dışındaki birçok görüntü yaması arka plan olarak kabul edilir.
- Yeni bir çerçeve verildiğinde, sınıflandırıcı, önceki konumun çevresindeki her piksel üzerinde çalıştırılır ve sınıflandırıcının puanı kayıt edilir.
- Nesnenin yeni konumu, paunın maksimum olduğu yerdir.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time 
import pandas as pd

OPENCV_OBJECT_TRACKERS =   {"csrt" : cv2.TrackerCSRT_create,
                            "kcf" : cv2.TrackerKCF_create,
                            "boosting" : cv2.legacy.TrackerBoosting_create,
                            "mil" : cv2.TrackerMIL_create,
                            "tld" : cv2.legacy.TrackerTLD_create,
                            "medianflow" : cv2.legacy.TrackerMedianFlow_create,
                            "mosse" : cv2.legacy.TrackerMOSSE_create}

tracker_name = "boosting"
tracker = OPENCV_OBJECT_TRACKERS[tracker_name]()
print(tracker)

<legacy_TrackerBoosting 0x7fd693387f30>


In [7]:
# grand truth aktarılması 
gt = pd.read_csv("gt_new.txt")
# video aktarılması
video = cv2.VideoCapture("MOT17-13-SDP.mp4")

In [13]:
initBB = None 
fps = 25 
frame_number =[]
f = 0
succes_track_frame_succes = 0 
start_time = time.time()

# video kayıt için video writer
writer = cv2.VideoWriter('boosting.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 25, (960, 540))

while True:
    ret, frame = video.read()
    if ret :
        frame = cv2.resize(frame, (960, 540))
        (H, W) = frame.shape[:2]

        #gt 
        car_gt = gt[gt.frame_no == f]
        if len(car_gt) != 0:
            x = car_gt.x.values[0]
            y = car_gt.y.values[0]
            w = car_gt.w.values[0]
            h = car_gt.h.values[0]

            center_x = car_gt.center_x.values[0]
            center_y = car_gt.center_y.values[0]

            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
            cv2.circle(frame, (center_x, center_y), 2, (0,255,255), -1)

            time.sleep(0.01)
            # video kayıt
            writer.write(frame)   

            frame_number.append(f)
            f = f + 1
    else :
        break

writer.release()       
            



In [5]:
import cv2

tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
tracker_type = tracker_types[0]

if tracker_type == 'BOOSTING':
    tracker = cv2.legacy.TrackerBoosting_create()
if tracker_type == 'MIL':
    tracker = cv2.TrackerMIL_create() 
if tracker_type == 'KCF':
    tracker = cv2.TrackerKCF_create() 
if tracker_type == 'TLD':
    tracker = cv2.legacy.TrackerTLD_create() 
if tracker_type == 'MEDIANFLOW':
    tracker = cv2.legacy.TrackerMedianFlow_create() 
if tracker_type == 'MOSSE':
    tracker = cv2.legacy.TrackerMOSSE_create()
if tracker_type == "CSRT":
    tracker = cv2.TrackerCSRT_create()

In [6]:
# Get the video file and read it
video = cv2.VideoCapture("MOT17-13-SDP.mp4")
ret, frame = video.read()

frame_height, frame_width = frame.shape[:2]
# Resize the video for a more convinient view
frame = cv2.resize(frame, [frame_width//2, frame_height//2])
# Initialize video writer to save the results
output = cv2.VideoWriter(f'{tracker_type}.avi', 
                         cv2.VideoWriter_fourcc(*'XVID'), 60.0, 
                         (frame_width//2, frame_height//2), True)
if not ret:
    print('cannot read the video')
# Select the bounding box in the first frame
bbox = cv2.selectROI(frame, False)
ret = tracker.init(frame, bbox)
# Start tracking
while True:
    ret, frame = video.read()
    frame = cv2.resize(frame, [frame_width//2, frame_height//2])
    if not ret:
        print('something went wrong')
        break
    timer = cv2.getTickCount()
    ret, bbox = tracker.update(frame)
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)
    if ret:
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else:
        cv2.putText(frame, "Tracking failure detected", (100,80), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
    cv2.putText(frame, tracker_type + " Tracker", (100,20), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
    cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
    cv2.imshow("Tracking", frame)
    output.write(frame)
    k = cv2.waitKey(1) & 0xff
    if k == 27 : break
        
video.release()
output.release()
cv2.destroyAllWindows()

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
